# Spectograms
The studied samples do not rely with a piano, a flute, a violin or a trumpet.

### 2.9 Propose an explanation for why the previous samples do not sound natural

It is just a period of a note, so we can not determine its frequence accurately.

### 2.10 Load, find frequency and play the sound
The files `PianoLa.wav` (Note that this one is `.wav`, it is not a typo), `TrumpetLa.aif`, `ViolinLa.aif` and `FluteLa.aif` correspond to the full recording of the sounds from which the previous period samples have been taken.

In [ ]:
# Import necessary libraries
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import sounddevice as sd

# List of necessary files to load
files = ['./TDSound/PianoLa.wav',
         './TDSound/FluteLa.aif',
         './TDSound/ViolinLa.aif',
         './TDSound/TrumpetLa.aif']

# Store sampling frequency as a float
# and amplitudes as an array of each file in:
frequencies = [0] * len(files)
amplitudes = [0] * len(files)

# For each file, store the sampling frequency and the amplitudes
for i in range(len(files)):
    amplitudes[i], frequencies[i] = sf.read(files[i], dtype='float32')   # It loads files and extracts info

# There are 2 channels of sound, we separate them
ch1_amplitudes = [0] * len(files)
ch2_amplitudes = [0] * len(files)
for i in range(len(files)):
    ch1_amplitudes[i] = [amplitudes[i][j][0] for j in range(len(amplitudes[i]))]
    ch2_amplitudes[i] = [amplitudes[i][j][1] for j in range(len(amplitudes[i]))]

# Plot the amplitudes of each file, for ensuring everything is ok
instruments = []    # List of instruments
for i in range(len(files)):
    file = files[i]
    instruments.append(file[10:-4])
    print('Sampling frequency of', instruments[i], 'file:', frequencies[i])

#### Is the sound closer to what you expect?

In [ ]:
playing = np.repeat(np.asarray(amplitudes), 1000, axis=1)
for i in range(len(amplitudes)):
    sd.play(playing[i, :], frequencies[i])
    status = sd.wait()